# Lemmatize PATSTATusing Spark NLP

In [1]:
from sparknlp.base import *
from sparknlp.annotator import *
import sparknlp
from pyspark.ml import Pipeline
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pathlib import Path

## 1. Read papers and concatenate the `title` and `paperAbstract` fields

In [2]:
%%time

# Loading patents table text fields, and concatenating them for lemmatization
patents = spark.sql("SELECT appln_id, appln_title, appln_abstract FROM parquet.`/export/ml4ds/IntelComp/Datalake/PATSTAT/2021_Autumn/patstat_appln.parquet`")
patents = patents.repartition(numPartitions=20000)
##For development purposes only
#patents = patents.sample(fraction=0.0001)

#Concatenate text fields to lemmatize
patents = (
    patents.withColumn("rawtext",F.concat_ws('. ', "appln_title", "appln_abstract"))
    .drop("appln_title")
    .drop("appln_abstract")
)

print('Number of patents before language filtering:', patents.count())

22/05/12 13:17:14 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/05/12 13:17:14 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/05/12 13:17:17 WARN metastore.ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
22/05/12 13:17:17 WARN metastore.ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore jarenas@192.168.148.225
22/05/12 13:17:17 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
22/05/12 13:17:17 WARN metastore.ObjectStore: Failed to get database parquet, returning NoSuchObjectException


Number of patents before language filtering: 114690034
CPU times: user 70.6 ms, sys: 17.8 ms, total: 88.4 ms
Wall time: 45.2 s


## 2. Filter patents abstracts that are not in English Language

In [3]:
%%time

#Pipeline for language detection
documentAssembler = DocumentAssembler() \
    .setInputCol("rawtext") \
    .setOutputCol("document")

languageDetector = LanguageDetectorDL.pretrained() \
    .setInputCols("document") \
    .setOutputCol("language")

pipeline = Pipeline() \
    .setStages([
      documentAssembler,
      languageDetector
    ])

#Apply language detection pipeline
patents = pipeline.fit(patents).transform(patents)
patents = (
    patents.filter(F.col("language.result")[0]=="en")
    .drop("language")
)

print('Number of patents in English:', patents.count())

ld_wiki_tatoeba_cnn_21 download started this may take some time.
Approximate size to download 7.1 MB
[ / ]ld_wiki_tatoeba_cnn_21 download started this may take some time.
Approximate size to download 7.1 MB
Download done! Loading the resource.
[ — ]

[ \ ]

[ | ]

[ / ]

2022-05-12 13:18:15.378688: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[OK!]


Number of patents in English: 76348837
CPU times: user 1.03 s, sys: 164 ms, total: 1.19 s
Wall time: 1h 33min 23s


## 3. Define and Run Lemmatization Pipeline

   - We work on documents created in Subsection 2
   - Sentence Detection and Tokenizer applied to detect tokens
   - Lemmatization is carried out
   - Stopwords are applied
   - Punctuation symbols are removed
   - Result is converted back from Spark NLP annotations to string format

In [4]:
%%time 

#Next, we carry out the lemmatization pipeline

sentenceDetector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(["token"]) \
    .setOutputCol("lemma")

stopWords = StopWordsCleaner() \
    .setInputCols(["lemma"]) \
    .setOutputCol("cleanlemma")

normalizer = Normalizer() \
    .setInputCols(["cleanlemma"]) \
    .setOutputCol("normalizedlemma") \
    .setLowercase(True) \
    .setCleanupPatterns(["""[^\w\d\s]"""])

finisher = Finisher() \
     .setInputCols(['normalizedlemma'])

pipeline = Pipeline() \
    .setStages([
      sentenceDetector,
      tokenizer,
      lemmatizer,
      stopWords,
      normalizer,
      finisher
])

#We apply pipeline and recover lemmas as string
patents = pipeline.fit(patents).transform(patents)

udf_back2str = F.udf(lambda x:' '.join(list(x)), StringType() )
patents = (
    patents.withColumn("lemmas",udf_back2str(F.col("finished_normalizedlemma")))
    .drop("finished_normalizedlemma")
)

#Show results of validation for n papers
#patents.show(n=10, truncate=120, vertical=True)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ]lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
Download done! Loading the resource.


[ — ]

[OK!]
CPU times: user 117 ms, sys: 13.2 ms, total: 130 ms
Wall time: 9.17 s


## 4. Save a table with `appln_id`, `rawtext` and `lemmas` to HDFS

In [5]:
%%time

#Save calculated lemmas to HDFS
dir_parquet = Path("/export/ml4ds/IntelComp/Datalake/PATSTAT/2021_Autumn")

patents.coalesce(1000).write.parquet(
    dir_parquet.joinpath(f"patstat_appln_NLP.parquet").as_posix(),
    mode="overwrite",
)

22/05/12 16:31:17 ERROR scheduler.TaskSchedulerImpl: Lost executor 22 on node25.cluster.tsc.uc3m.es: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/05/12 16:31:17 WARN scheduler.TaskSetManager: Lost task 59.0 in stage 13.0 (TID 41300) (node25.cluster.tsc.uc3m.es executor 22): ExecutorLostFailure (executor 22 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/05/12 16:31:17 WARN scheduler.TaskSetManager: Lost task 209.0 in stage 13.0 (TID 41450) (node25.cluster.tsc.uc3m.es executor 22): ExecutorLostFailure (executor 22 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/05/12 16:31:17 WARN scheduler.TaskSetManager: Lost task

CPU times: user 1.71 s, sys: 362 ms, total: 2.07 s
Wall time: 2h 1min 47s


## 5. Optional: Check that the generated table looks OK

In [12]:
%%time

#Test that the saved table is correct
patents = spark.sql("SELECT * FROM parquet.`/export/ml4ds/IntelComp/Datalake/PATSTAT/2021_Autumn/patstat_appln_NLP.parquet`")
print('Number of lemmatized patents:', patents.count())
patents.show(n=10, truncate=120, vertical=True)

22/03/22 18:00:34 WARN metastore.ObjectStore: Failed to get database parquet, returning NoSuchObjectException


Number of lemmatized patents: 76348837
-RECORD 0----------------------------------------------------------------------------------------------------------------------------
 appln_id | 496467995                                                                                                                
 lemmas   | medicinal material sun room utility model disclose medicinal material sun room include room body glass cover equip ot... 
-RECORD 1----------------------------------------------------------------------------------------------------------------------------
 appln_id | 4882879                                                                                                                  
 lemmas   | high expression locus vector based ferritin heavy chain gene locus high expression locus vector base part ferritin he... 
-RECORD 2----------------------------------------------------------------------------------------------------------------------------
 appln_id | 7630854    